In [1]:
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
import itertools

from keras.models import load_model
from keras.utils import np_utils

import tensorflow as tf
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb
import time

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
#         visible_device_list="1", 
        allow_growth=True
    )
)

set_session(tf.Session(config=config))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load data
df = pd.concat([
    pd.read_csv('../data/feature_selection_positive.csv', index_col=0),
    pd.read_csv('../data/decomp_pos.csv', index_col=0).drop('Subclass', axis=1)
], axis=1)

In [3]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [4]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [5]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [6]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# append accuracy to list
t = []
t.append(clf.score(X_test, y_test))
pickle.dump(clf, open('../model/rf_pos_fs+decomp.sav', 'wb'))

In [7]:
# setting params
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [8]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

pickle.dump(model, open('../model/xgb_pos_fs+decomp.sav', 'wb'))
t.append(model.score(X_test, y_test))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [9]:
# define and fit 
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)
pickle.dump(gbm, open('../model/lgbm_pos_fs+decomp.sav', 'wb'))
t.append(gbm.score(X_test, y_test))

[1]	valid_0's multi_logloss: 1.29067
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.21059
[3]	valid_0's multi_logloss: 1.14259
[4]	valid_0's multi_logloss: 1.08432
[5]	valid_0's multi_logloss: 1.0303
[6]	valid_0's multi_logloss: 0.985829
[7]	valid_0's multi_logloss: 0.943392
[8]	valid_0's multi_logloss: 0.903674
[9]	valid_0's multi_logloss: 0.866989
[10]	valid_0's multi_logloss: 0.836213
[11]	valid_0's multi_logloss: 0.808068
[12]	valid_0's multi_logloss: 0.782105
[13]	valid_0's multi_logloss: 0.759796
[14]	valid_0's multi_logloss: 0.738561
[15]	valid_0's multi_logloss: 0.720694
[16]	valid_0's multi_logloss: 0.70385
[17]	valid_0's multi_logloss: 0.685661
[18]	valid_0's multi_logloss: 0.668469
[19]	valid_0's multi_logloss: 0.653594
[20]	valid_0's multi_logloss: 0.640255
[21]	valid_0's multi_logloss: 0.627174
[22]	valid_0's multi_logloss: 0.616858
[23]	valid_0's multi_logloss: 0.604124
[24]	valid_0's multi_logloss: 0.594588
[25]	valid_0's mult

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_pos_fs+decomp.h5')
t.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 1s 1ms/step - loss: 1.8758 - acc: 0.2478 - val_loss: 1.4750 - val_acc: 0.2143
Epoch 2/100
1340/1340 [==============================] - 0s 96us/step - loss: 1.8067 - acc: 0.2627 - val_loss: 1.4211 - val_acc: 0.2768
Epoch 3/100
1340/1340 [==============================] - 0s 101us/step - loss: 1.6896 - acc: 0.2590 - val_loss: 1.3850 - val_acc: 0.3065
Epoch 4/100
1340/1340 [==============================] - 0s 93us/step - loss: 1.6366 - acc: 0.2955 - val_loss: 1.3614 - val_acc: 0.3363
Epoch 5/100
1340/1340 [==============================] - 0s 102us/step - loss: 1.6427 - acc: 0.2925 - val_loss: 1.3317 - val_acc: 0.4167
Epoch 6/100
1340/1340 [==============================] - 0s 104us/step - loss: 1.5719 - acc: 0.3224 - val_loss: 1.3131 - val_acc: 0.3839
Epoch 7/100
1340/1340 [==============================] - 0s 105us/step - loss: 1.5583 - acc: 0.3231 - val_loss: 1.3068 - val_acc: 0.434

1340/1340 [==============================] - 0s 101us/step - loss: 0.5587 - acc: 0.7634 - val_loss: 0.9357 - val_acc: 0.6042
Epoch 61/100
1340/1340 [==============================] - 0s 93us/step - loss: 0.5714 - acc: 0.7657 - val_loss: 0.9354 - val_acc: 0.5982
Epoch 62/100
1340/1340 [==============================] - 0s 87us/step - loss: 0.5781 - acc: 0.7806 - val_loss: 0.8834 - val_acc: 0.6220
Epoch 63/100
1340/1340 [==============================] - 0s 80us/step - loss: 0.5300 - acc: 0.7873 - val_loss: 0.8564 - val_acc: 0.6399
Epoch 64/100
1340/1340 [==============================] - 0s 81us/step - loss: 0.5706 - acc: 0.7731 - val_loss: 0.8803 - val_acc: 0.6339
Epoch 65/100
1340/1340 [==============================] - 0s 84us/step - loss: 0.5504 - acc: 0.7828 - val_loss: 0.9065 - val_acc: 0.6220
Epoch 66/100
1340/1340 [==============================] - 0s 95us/step - loss: 0.5091 - acc: 0.8015 - val_loss: 0.9131 - val_acc: 0.6280
Epoch 67/100
1340/1340 [=============================

In [11]:
f = []
f.append(t)

In [12]:
f

[[0.7648809523809523,
  0.8095238095238095,
  0.8303571428571429,
  0.6458333333333334]]

In [13]:
# load data
df = pd.concat([
    pd.read_csv('../data/feature_selection_negative.csv', index_col=0),
    pd.read_csv('../data/decomp_neg.csv', index_col=0).drop('Subclass', axis=1)
], axis=1)

In [14]:
df

,Subclass,HCD35_311.3,HCD35_167.07,HCD35_177.05,HCD35_115.0,HCD35_184.02,HCD35_141.09,HCD35_98.95,HCD35_197.04,HCD35_57.03,...,NMF_0_6,NMF_1_6,NMF_2_6,NMF_3_6,NMF_4_6,FAG_0_6,FAG_1_6,FAG_2_6,FAG_3_6,FAG_4_6
0,Carbohydrates and carbohydrate conjugates,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,9.838692e+04,...,441.189819,6.420388,0.390083,174.402759,0.000000,406.022142,7.421322e+06,0.000000e+00,0.000000e+00,2.657055e+06
1,Carbohydrates and carbohydrate conjugates,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.577014e+05,...,150.440454,3.931254,0.165852,8.008466,1.725556,835.646881,2.419239e+06,0.000000e+00,0.000000e+00,0.000000e+00
2,Diterpenoids,15597593.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,14.008920,0.461140,0.110324,223.861752,0.086405,518.351044,2.988356e+05,0.000000e+00,0.000000e+00,3.519269e+06
3,Diterpenoids,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,5.847812e+05,...,9.416152,0.000000,0.142362,96.100760,2.059125,5688.164779,0.000000e+00,0.000000e+00,0.000000e+00,1.365321e+06
4,Diterpenoids,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,807.499515,27.305535,2.863465,10.306684,0.000000,75.498157,1.725016e+07,0.000000e+00,0.000000e+00,0.000000e+00
5,Terpene lactones,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.133839e+06,...,15.429370,1.707767,0.039210,1.035661,0.676517,712.743784,7.938673e+03,0.000000e+00,0.000000e+00,0.000000e+00
6,Terpene lactones,0.0,0.000000,8.470924e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,7.941203e+04,...,2.297116,2.845556,0.018110,1.840802,0.223223,102.085432,3.274544e+04,1.550901e+04,0.000000e+00,1.778840e+04
7,Terpene lactones,0.0,0.000000,0.000000e+00,0.000000e+00,59732.203125,0.000000,0.000000,0.000000,5.084775e+04,...,0.000000,0.000000,0.000000,836.916580,0.000000,403.092978,0.000000e+00,0.000000e+00,0.000000e+00,1.339061e+07
8,Terpene glycosides,0.0,196225.484375,1.579421e+05,0.000000e+00,0.000000,0.000000,0.000000,0.000000,3.045320e+05,...,720.461091,26.968811,0.788273,8.519395,5.347679,3325.326421,1.164435e+07,2.040203e+05,0.000000e+00,0.000000e+00
9,Terpene lactones,0.0,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,4.176409,6.776744,0.000000,22.081613,394.637076,13090.178215,0.000000e+00,0.000000e+00,0.000000e+00,1.115718e+05


In [15]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [16]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [17]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [18]:
# define and fit
clf = rf()
clf = clf.fit(X_train, y_train)

# append accuracy to list
t = []
t.append(clf.score(X_test, y_test))

pickle.dump(clf, open('../model/rf_ng_fs+decomp.sav', 'wb'))

In [19]:
# setting params
params = {}
params['device'] = 'gpu'
params['gpu_id'] = 1
params['updater'] = 'grow_gpu_hist'

In [20]:
# define and fit
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

pickle.dump(model, open('../model/xgb_ng_fs+decomp.sav', 'wb'))
t.append(model.score(X_test, y_test))

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [21]:
# define and fit 
gbm = lgb.LGBMClassifier(
    objective='multiclass',
    device = 'gpu'
)

gbm.fit(
    X_train, 
    y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=5
)

pickle.dump(gbm, open('../model/lgbm_ng_fs+decomp.sav', 'wb'))
t.append(gbm.score(X_test, y_test))

[1]	valid_0's multi_logloss: 1.28963
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.20936
[3]	valid_0's multi_logloss: 1.14064
[4]	valid_0's multi_logloss: 1.08152
[5]	valid_0's multi_logloss: 1.03213
[6]	valid_0's multi_logloss: 0.985415
[7]	valid_0's multi_logloss: 0.945303
[8]	valid_0's multi_logloss: 0.91431
[9]	valid_0's multi_logloss: 0.888262
[10]	valid_0's multi_logloss: 0.854722
[11]	valid_0's multi_logloss: 0.834649
[12]	valid_0's multi_logloss: 0.807642
[13]	valid_0's multi_logloss: 0.787779
[14]	valid_0's multi_logloss: 0.770556
[15]	valid_0's multi_logloss: 0.755186
[16]	valid_0's multi_logloss: 0.741407
[17]	valid_0's multi_logloss: 0.727504
[18]	valid_0's multi_logloss: 0.717873
[19]	valid_0's multi_logloss: 0.706302
[20]	valid_0's multi_logloss: 0.697141
[21]	valid_0's multi_logloss: 0.689405
[22]	valid_0's multi_logloss: 0.680429
[23]	valid_0's multi_logloss: 0.673084
[24]	valid_0's multi_logloss: 0.663289
[25]	valid_0's mul

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [22]:
y_pred = gbm.predict(X_test)

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [23]:
tf.set_random_seed(42)

# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_ng_fs+decomp.h5')
t.append(model.evaluate(X_test, y_test_for_keras, verbose=0)[1])

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 1s 3ms/step - loss: 2.1088 - acc: 0.2493 - val_loss: 1.2895 - val_acc: 0.4468
Epoch 2/100
373/373 [==============================] - 0s 50us/step - loss: 1.8163 - acc: 0.2869 - val_loss: 1.2660 - val_acc: 0.4574
Epoch 3/100
373/373 [==============================] - 0s 67us/step - loss: 1.7722 - acc: 0.2788 - val_loss: 1.3590 - val_acc: 0.4362
Epoch 4/100
373/373 [==============================] - 0s 64us/step - loss: 1.7268 - acc: 0.3485 - val_loss: 1.4389 - val_acc: 0.4468
Epoch 5/100
373/373 [==============================] - 0s 65us/step - loss: 1.8073 - acc: 0.3190 - val_loss: 1.4684 - val_acc: 0.4574
Epoch 6/100
373/373 [==============================] - 0s 62us/step - loss: 1.7605 - acc: 0.3807 - val_loss: 1.4443 - val_acc: 0.4574
Epoch 7/100
373/373 [==============================] - 0s 65us/step - loss: 1.7344 - acc: 0.3298 - val_loss: 1.4014 - val_acc: 0.4787
Epoch 8/100
373/37

In [24]:
f

[[0.7648809523809523,
  0.8095238095238095,
  0.8303571428571429,
  0.6458333333333334]]

In [25]:
f.append(t)

In [26]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'XGBoost', 'LightGBM', 'Keras'], 
    index=['Feature Selection + Decomposition_Positive', 'Feature Selection + Decomposition_Negative']
)

,Random Forest,XGBoost,LightGBM,Keras
Feature Selection + Decomposition_Positive,0.764881,0.809524,0.830357,0.645833
Feature Selection + Decomposition_Negative,0.723404,0.734043,0.776596,0.563830


In [27]:
pd.DataFrame(
    f, 
    columns=['Random Forest', 'XGBoost', 'LightGBM', 'Keras'], 
    index=['Feature Selection + Decomposition_Positive', 'Feature Selection + Decomposition_Negative']
).T.to_csv('../result/Feature Selection + Decomposition.csv')